In [27]:
import pandas as pd

In [28]:
df_detail = pd.read_excel("inputData/details.xlsx",usecols="C,D",names=["department","welfare_level"])

In [29]:
df_detail.head()

,department,welfare_level
0,高层管理,1
1,高层管理,1
2,高层管理,1
3,文化产业部,2
4,文化产业部,2


In [30]:
df_welfare = pd.read_excel("inputData/welfare.xlsx",usecols="A,B",names=["price","content"])

In [31]:
df_welfare.head()

,price,content
0,100元,怡青源月饼8*100g
1,NaN,大米
2,NaN,后腿老腊肉
3,NaN,NaN
4,NaN,NaN


In [32]:
df_welfare = df_welfare[df_welfare.content.isnull()==False]

In [33]:
df_welfare = df_welfare.dropna(axis=0,how="all")

In [34]:
df_welfare.head()

,price,content
0,100元,怡青源月饼8*100g
1,NaN,大米
2,NaN,后腿老腊肉
5,200元,怡青源月饼8*100g
6,NaN,大米


In [35]:
df_welfare = df_welfare.fillna(method="ffill")
df_welfare

,price,content
0,100元,怡青源月饼8*100g
1,100元,大米
2,100元,后腿老腊肉
5,200元,怡青源月饼8*100g
6,200元,大米
7,200元,猕猴桃
8,200元,龙大花生油
9,200元,面粉
13,300元,怡青源月饼8*100g
14,300元,大米


In [38]:
def convert_level(x):
    dct = {"100元":4,"200元":3,"300元":2,"500元":1}
    return dct[x]
df_welfare["level"] = df_welfare.price.apply(lambda x:convert_level(x))
df_welfare.drop("price",inplace=True,axis=1)

In [41]:
df_welfare.rename(columns={"level":"welfare_level"},inplace=True)

In [42]:
summary = df_detail.merge(df_welfare)

In [44]:
summary["quanity"] = 1

In [45]:
summary.head()

,department,welfare_level,content,quanity
0,高层管理,1,喜盛隆月饼,1
1,高层管理,1,胡姬花花生油,1
2,高层管理,1,黄河口大闸蟹,1
3,高层管理,1,大米,1
4,高层管理,1,后腿老腊肉,1


In [46]:
gp_dp = summary.groupby(["department","content"]).sum()
del gp_dp["welfare_level"]

In [48]:
department = df_detail.department.unique()
department

array(['高层管理', '文化产业部', '董秘办', '秘书处', '电商政策审核部', '市场部', '企划中心', '商学院',
       '财务中心', '档案部', '综合管理中心', '行政中心', '人事中心', '品牌中心', '策划中心', '技术中心',
       '招商部', '产品管理中心采购部', '产品管理中心产品部', '产品管理中心物流部', '产品管理中心客服部', '审计中心',
       '运营中心', '公共关系', '操盘', '太空舱项目', '智能板块', '健康板块'], dtype=object)

In [50]:
meterials = df_welfare.content.unique()
meterials

array(['怡青源月饼8*100g', '大米', '后腿老腊肉', '猕猴桃', '龙大花生油', '面粉', '酱醋精装组合',
       '胡姬花花生油', '茶叶', '五花老腊肉', '喜盛隆月饼', '黄河口大闸蟹'], dtype=object)

In [51]:
result = pd.DataFrame(index=department,columns=meterials)

In [53]:
for i in department:
    for j in meterials:
        try:
            result.loc[i][j] = int(gp_dp.loc[i].loc[j])
        except KeyError:
            result.loc[i][j] = 0

In [54]:
result

,怡青源月饼8*100g,大米,后腿老腊肉,猕猴桃,龙大花生油,面粉,酱醋精装组合,胡姬花花生油,茶叶,五花老腊肉,喜盛隆月饼,黄河口大闸蟹
高层管理,0,3,3,0,0,0,3,3,0,3,3,3
文化产业部,25,25,11,14,14,14,2,2,2,2,0,0
董秘办,3,3,0,3,3,3,0,0,0,0,0,0
秘书处,2,2,2,0,0,0,1,1,1,1,0,0
电商政策审核部,2,2,1,1,1,1,1,1,1,1,0,0
市场部,5,5,4,1,1,1,2,2,2,2,0,0
企划中心,2,2,1,1,1,1,1,1,1,1,0,0
商学院,2,2,0,2,2,2,0,0,0,0,0,0
财务中心,14,14,5,9,9,9,3,3,3,3,0,0
档案部,4,4,1,3,3,3,1,1,1,1,0,0


In [55]:
gp_con = summary.groupby("content").sum()

In [56]:
del gp_con["welfare_level"]

In [59]:
total = pd.DataFrame(index=["总计"],columns=meterials)

In [60]:
total

,怡青源月饼8*100g,大米,后腿老腊肉,猕猴桃,龙大花生油,面粉,酱醋精装组合,胡姬花花生油,茶叶,五花老腊肉,喜盛隆月饼,黄河口大闸蟹
总计,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
result = result.append(total)

In [62]:
for i in meterials:
    result.loc["总计"][i] = int(gp_con.loc[i])

In [63]:
result

,怡青源月饼8*100g,大米,后腿老腊肉,猕猴桃,龙大花生油,面粉,酱醋精装组合,胡姬花花生油,茶叶,五花老腊肉,喜盛隆月饼,黄河口大闸蟹
高层管理,0,3,3,0,0,0,3,3,0,3,3,3
文化产业部,25,25,11,14,14,14,2,2,2,2,0,0
董秘办,3,3,0,3,3,3,0,0,0,0,0,0
秘书处,2,2,2,0,0,0,1,1,1,1,0,0
电商政策审核部,2,2,1,1,1,1,1,1,1,1,0,0
市场部,5,5,4,1,1,1,2,2,2,2,0,0
企划中心,2,2,1,1,1,1,1,1,1,1,0,0
商学院,2,2,0,2,2,2,0,0,0,0,0,0
财务中心,14,14,5,9,9,9,3,3,3,3,0,0
档案部,4,4,1,3,3,3,1,1,1,1,0,0
